In [1]:
import os
import pandas as pd
import numpy as np
import json
import pandas as pd
from tqdm import tqdm
from collections import Counter
import shutil

In [2]:
HAMdatafold = './SkinLesionDatasets/HAM/'
MSKdatafold = './SkinLesionDatasets/MSK/'
SONdatafold = './SkinLesionDatasets/SON/'
UDAdatafold = './SkinLesionDatasets/UDA/'
BCNdatafold = './SkinLesionDatasets/BCN/'
VIEdatafold = './SkinLesionDatasets/VIE/'
OTHdatafold = './SkinLesionDatasets/OTH/'
ISICdownload = './ISICdownload/Data/'

In [3]:
ISICdownload_Descriptions = ISICdownload + '/Images/metadata.csv'
ISICdownload_meta = ISICdownload + 'meta_v1.csv'
ISICdownload_Images = ISICdownload + 'Images/'

In [4]:
existingdescriptionfiles = pd.read_csv(ISICdownload_Descriptions)
existingmetav1 = pd.read_csv(ISICdownload_meta)
existingimagefiles = os.listdir(ISICdownload_Images)

/well/win-fmrib-analysis/users/gqu790/conda/skylake/envs/skinclassifier/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (8,13,14,15,17,18,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
len(existingimagefiles)

76298

In [6]:
len(existingdescriptionfiles)

76295

In [7]:
len(existingmetav1)

69445

In [8]:
datasdigno = Counter(existingdescriptionfiles['diagnosis'])

In [9]:
datasdigno

Counter({'melanoma': 6701,
         'nevus': 31626,
         'seborrheic keratosis': 1725,
         'actinic keratosis': 1011,
         'basal cell carcinoma': 3869,
         'solar lentigo': 446,
         'squamous cell carcinoma': 879,
         'dermatofibroma': 370,
         'vascular lesion': 348,
         'lichenoid keratosis': 93,
         'lentigo NOS': 222,
         'atypical melanocytic proliferation': 99,
         'verruca': 6,
         'clear cell acanthoma': 1,
         'angiofibroma or fibrous papule': 2,
         'scar': 6,
         'angioma': 16,
         'atypical spitz tumor': 1,
         nan: 27438,
         'AIMP': 22,
         'neurofibroma': 7,
         'lentigo simplex': 31,
         'acrochordon': 5,
         'angiokeratoma': 1,
         'other': 25,
         'cafe-au-lait macule': 2,
         'pigmented benign keratosis': 1339,
         'melanoma metastasis': 4})

In [10]:
dign_list_HAM = ['melanoma', 'nevus', 'actinic keratosis', 'squamous cell carcinoma', 'basal cell carcinoma', 
                 'dermatofibroma', 'vascular lesion',
                'seborrheic keratosis', 'solar lentigo', 'lichenoid keratosis', 'pigmented benign keratosis']

In [11]:
''' following the HAM dataset descriptions (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T)
akiec: actinic keratoses / squamous cell carcinoma
bcc: basal cell carcinoma
bkl: pigmented benign keratosis / seborrheic keratosis / solar lentigo / lichenoid keratosis
df: dermatofibroma
mel: melanoma
nv: nevus
vasc: vascular lesions
'''

' following the HAM dataset descriptions (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T)\nakiec: actinic keratoses / squamous cell carcinoma\nbcc: basal cell carcinoma\nbkl: pigmented benign keratosis / seborrheic keratosis / solar lentigo / lichenoid keratosis\ndf: dermatofibroma\nmel: melanoma\nnv: nevus\nvasc: vascular lesions\n'

In [12]:
source_list = ['HAM', 'BCN_200', 'ISIC_', 'MSK', 'SON', 'UDA']

In [13]:
len(existingmetav1.index[existingmetav1['ID'] == 'ISIC_6967412'])

0

In [14]:
len(existingmetav1.index[existingmetav1['ID'] == 'ISIC_0000004'])

1

In [15]:
existingmetav1['Dataset'][existingmetav1.index[existingmetav1['ID'] == 'ISIC_0000004'].tolist()[0]]

'UDA-1'

In [16]:
existingdescriptionfiles['diagnosis'][0] not in dign_list_HAM

False

In [17]:
datas = Counter(existingmetav1['Dataset'])

In [18]:
datas

Counter({'UDA-1': 557,
         'SONIC': 9251,
         'MSK-2': 1535,
         'MSK-1': 1100,
         'HAM10000': 10015,
         'BCN_20000': 12413,
         'BCN_2020_Challenge': 7311,
         'Brisbane ISIC Challenge 2020': 8449,
         'ISIC 2020 Challenge - MSKCC contribution': 11108,
         'ISIC_2020_Vienna_part2': 2143,
         'Sydney (MIA / SMDC) 2020 ISIC challenge contribution': 1884,
         'ISIC_2020_Vienna_part_1': 2231,
         'MSK-4': 947,
         'MSK-3': 225,
         'MSK-5': 111,
         '2018 JID Editorial Images': 100,
         'UDA-2': 60,
         'Dermoscopedia (CC-BY)': 5})

In [19]:
existingdescriptionfiles['diagnosis'][existingdescriptionfiles.index[existingdescriptionfiles['isic_id'] == 'ISIC_0024329'].tolist()[0]]

'squamous cell carcinoma'

In [20]:
for count in tqdm(range(len(existingdescriptionfiles))):
#     print(count)
    
    ID = existingdescriptionfiles['isic_id'][count]
    if existingdescriptionfiles['diagnosis'][count] not in dign_list_HAM:
        continue
    
    # get the dignosis information
    if existingdescriptionfiles['diagnosis'][count] == 'actinic keratosis':
        diagnosisinfo = 'akiec'
    if existingdescriptionfiles['diagnosis'][count] == 'squamous cell carcinoma':
        diagnosisinfo = 'akiec'
    if existingdescriptionfiles['diagnosis'][count] == 'basal cell carcinoma':
        diagnosisinfo = 'bcc'
    if existingdescriptionfiles['diagnosis'][count] == 'pigmented benign keratosis':
        diagnosisinfo = 'bkl'
    if existingdescriptionfiles['diagnosis'][count] == 'seborrheic keratosis':
        diagnosisinfo = 'bkl'
    if existingdescriptionfiles['diagnosis'][count] == 'solar lentigo':
        diagnosisinfo = 'bkl'
    if existingdescriptionfiles['diagnosis'][count] == 'lichenoid keratosis':
        diagnosisinfo = 'bkl'
    if existingdescriptionfiles['diagnosis'][count] == 'dermatofibroma':
        diagnosisinfo = 'df'
    if existingdescriptionfiles['diagnosis'][count] == 'melanoma':
        diagnosisinfo = 'mel'
    if existingdescriptionfiles['diagnosis'][count] == 'nevus':
        diagnosisinfo = 'nv'
    if existingdescriptionfiles['diagnosis'][count] == 'vascular lesion':
        diagnosisinfo = 'vasc'
        
    if len(existingmetav1.index[existingmetav1['ID'] == ID]) > 0:
    
        datasetinfo = existingmetav1['Dataset'][existingmetav1.index[existingmetav1['ID'] == ID].tolist()[0]]

        # get the dataset information
        flag = 0
        if datasetinfo[:3] == 'HAM':
            outputfolder = HAMdatafold + diagnosisinfo
            flag = 1
        if datasetinfo[:7] == 'BCN_200':
            outputfolder = BCNdatafold + diagnosisinfo
            flag = 1
        if datasetinfo[:5] == 'ISIC_':
            outputfolder = VIEdatafold + diagnosisinfo
            flag = 1
        if datasetinfo[:3] == 'MSK':
            outputfolder = MSKdatafold + diagnosisinfo
            flag = 1
        if datasetinfo[:3] == 'SON':
            outputfolder = SONdatafold + diagnosisinfo
            flag = 1
        if datasetinfo[:3] == 'UDA':
            outputfolder = UDAdatafold + diagnosisinfo
            flag = 1
        if flag == 0:
            outputfolder = OTHdatafold + diagnosisinfo
    else:
        outputfolder = OTHdatafold + diagnosisinfo
    
    if os.path.isdir(outputfolder) == False:
        os.makedirs(outputfolder)
    
    src = ISICdownload_Images + ID + '.JPG'
    dst = outputfolder + '/' + ID + '.JPG'
    shutil.copyfile(src, dst)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76295/76295 [18:52<00:00, 67.37it/s]
